In [3]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series
from sklearn.linear_model import LinearRegression

from pipeline.parse_raw_df import pre_process_df_with_date_time, pre_process_df_with_date
from datatools import data_quantization, extract_market_data
from pipeline import Dataset, N_train_days, N_test_days, N_asset, N_timeslot
from pipeline.backtest import cross_validation, evaluation_for_submission
from visualization.metric import plot_performance

from tqdm.auto import trange
from qids_lib import QIDS

In [5]:
dataset = Dataset.load('../../data/parsed')
qids = QIDS(path_prefix='../../')

In [6]:
m_intraday_df = extract_market_data(dataset.market)
df = pd.concat([dataset.fundamental, m_intraday_df], axis=1)
df_with_return = pd.concat([dataset.fundamental, m_intraday_df, dataset.ref_return], axis=1).dropna()

In [21]:
features = ['turnoverRatio', 'transactionAmount', 'pe_ttm', 'pe', 'pb', 'ps', 'pcf']

In [9]:
def linear_model(X, y):
    reg = LinearRegression().fit(X, y)
    return reg

In [18]:
performance = cross_validation(linear_model, features, df_with_return, n_splits=997, lookback_window=200)

  0%|          | 0/997 [00:00<?, ?it/s]

In [19]:
plt.figure()
plot_performance(performance, metrics_selected=['train_r2', 'val_cum_pearson'])
plt.show()

<IPython.core.display.Javascript object>

The ending score for metric train_r2 is: 3.0795e-03
The ending score for metric val_cum_pearson is: 3.4290e-02


In [22]:
performance = evaluation_for_submission(linear_model, features, dataset, df, qids, lookback_window=200)

  0%|          | 0/700 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
plt.figure()
plot_performance(performance, metrics_selected=['train_r2', 'test_cum_r2', 'test_cum_pearson'])
plt.show()

<IPython.core.display.Javascript object>

The ending score for metric train_r2 is: 3.1536e-03
The ending score for metric test_cum_r2 is: nan
The ending score for metric test_cum_pearson is: nan


In [25]:
m_before = pd.concat((
    m_intraday_df['avg_price'].shift(54*i).rename(f'avg_price_{i}before') for i in range(10)
), axis=1)

In [26]:
m_before

avg_price_0before  avg_price_1before  avg_price_2before  \
day  asset                                                            
1    0              24.553713                NaN                NaN   
     1              16.169781                NaN                NaN   
     2               9.059178                NaN                NaN   
     3               9.124460                NaN                NaN   
     4               5.054415                NaN                NaN   
...                       ...                ...                ...   
1000 49             13.844115          13.926098          13.912934   
     50             11.271970          11.325441          11.220278   
     51              3.576689           3.677312           3.743402   
     52             28.328126          28.434445          28.845564   
     53              9.935985          10.024707           9.821027   

            avg_price_3before  avg_price_4before  avg_price_5before  \
day  asset                                                            
1    0                    NaN                NaN                NaN   
     1                    NaN                NaN                NaN   
     2                    NaN                NaN                NaN   
     3                    NaN                NaN                NaN   
     4                    NaN                NaN                NaN   
...                       ...                ...                ...   
1000 49             14.217612          14.260748          14.268627   
     50             11.405730          10.837950          10.469909   
     51              3.894224           3.922678           3.983210   
     52             28.892518          28.848403          28.557741   
     53              9.851847           9.444299           9.346187   

            avg_price_6before  avg_price_7before  avg_price_8before  \
day  asset                                                            
1    0                    NaN                NaN                NaN   
     1                    NaN                NaN                NaN   
     2                    NaN                NaN                NaN   
     3                    NaN                NaN                NaN   
     4                    NaN                NaN                NaN   
...                       ...                ...                ...   
1000 49             14.318401          14.293341          14.462391   
     50             10.271398          10.255132          10.547572   
     51              3.965433           3.975151           3.934831   
     52             28.663267          28.780066          29.089705   
     53              9.330414           9.376600           9.531540   

            avg_price_9before  
day  asset                     
1    0                    NaN  
     1                    NaN  
     2                    NaN  
     3                    NaN  
     4                    NaN  
...                       ...  
1000 49             14.455877  
     50             10.335761  
     51              3.953280  
     52             28.155794  
     53              9.308103  

[54000 rows x 10 columns]

In [27]:
df_with_return_before = pd.concat([dataset.fundamental, m_before, dataset.ref_return], axis=1).dropna()
features = [f'avg_price_{i}before' for i in range(10)]

In [33]:
performance = cross_validation(linear_model, features, df_with_return_before, n_splits=987, lookback_window=500)

  0%|          | 0/987 [00:00<?, ?it/s]

In [34]:
plt.figure()
plot_performance(performance, metrics_selected=['train_r2', 'val_cum_r2', 'val_cum_pearson'])
plt.show()

<IPython.core.display.Javascript object>

The ending score for metric train_r2 is: 3.8672e-04
The ending score for metric val_cum_r2 is: -2.0053e-02
The ending score for metric val_cum_pearson is: 1.5919e-02


In [ ]:
performance = evaluation_for_submission(linear_model, features, df_with_return_before, n_splits=987, lookback_window=500)